In [1]:
from my_libs import * 

/home/ken/.local/lib/python2.7/site-packages/pyfolio/pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'


In [2]:
api_key = "0fb7208227mshc3a081958186ecfp1679c5jsn018ab7050406"

In [3]:
city = "Pittsburg"

# Get Sold

In [4]:
table_name = "sold_list"


mongod = mongo()

try:
    existing = mongod.conn.get_data("select * from Housing.%s "%table_name)

#     count = len(existing)
except:
    count = 0
    existing = pd.DataFrame()
count = 0

while count < 2000:


    querystring = {
        "sort":"sold_date",
        "city":city,
        "offset":count,
        "state_code":"CA",
        "limit":200,
        "radius":20


    }

    url = "https://realtor.p.rapidapi.com/properties/v2/list-sold"


    headers = {
        'x-rapidapi-host': "realtor.p.rapidapi.com",
        'x-rapidapi-key': api_key
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    listing = response.json()

    listing= pd.DataFrame(listing["properties"])

    listing = listing[["property_id","listing_id","address","baths","baths_full","beds",\
                       "building_size","prop_type","last_update","list_date","lot_size",\
                        "rdc_web_url","year_built","price","page_no"]]

    new_listing = pd.DataFrame()
    address = pd.DataFrame()
    building = pd.DataFrame()
    lot = pd.DataFrame()
    for i in listing.index:
        temp = listing.loc[i]
        if temp["lot_size"] == temp["lot_size"]:
            lot = lot.append(pd.DataFrame(temp["lot_size"],index =[temp.name]))
        temp = temp.drop("lot_size")
        try:
            del temp["address"]["neighborhoods"]
        except: 
            pass
        if temp["address"] == temp["address"]:
            address = address.append(pd.DataFrame(temp["address"],index =[temp.name]))
        temp = temp.drop("address")
        if temp["building_size"] == temp["building_size"]:
            building = building.append(pd.DataFrame(temp["building_size"],index =[temp.name]))
        temp = temp.drop("building_size")

        new_listing = new_listing.append(temp)
    new_listing = new_listing.join(address).join(building,rsuffix="_building").join(lot,rsuffix="_lot")

    def format_date(data):
        if data is None:
            return None
        try:
            return datetime.strptime(data,"%Y-%m-%dT%H:%M:%SZ")
        except:
            return datetime.strptime(data,"%Y-%m-%d")

    new_listing.last_update = new_listing.last_update.apply(format_date)

    new_listing.list_date = new_listing.list_date.apply(format_date)
    
    new_listing["Refresh_Date"] = datetime.now()
    ### append new listing to existing  
    mongod = mongo()
    exist_count = len(existing)
    existing = existing.append(new_listing).drop_duplicates("property_id")

    ### update the table
    mongod = mongo()
    try:
        mongod.conn.conn.cursor().execute("drop table Housing.%s"%table_name)
    except:
        pass
    mongod.conn.to_sql(existing,database="Housing",table_name=table_name,if_exists="append")
    # get new count   
    
    if exist_count == len(existing):
        break
    
#     count = len(existing)

    count += 200
    
# get new existing table
#     existing = mongod.conn.get_data("select * from Housing.%s "%table_name)

Connection Successful


/home/ken/.local/lib/python2.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Connection Successful
Connection Successful
If no error message, task completed
Connection Successful
Connection Successful
If no error message, task completed
Connection Successful
Connection Successful
If no error message, task completed
Connection Successful
Connection Successful
If no error message, task completed
Connection Successful
Connection Successful
If no error message, task completed
Connection Successful
Connection Successful
If no error message, task completed
Connection Successful
Connection Successful
If no error message, task completed
Connection Successful
Connection Successful
If no error message, task completed
Connection Successful
Connection Successful
If no error message, task completed
Connection Successful
Connection Successful
If no error message, task completed


# Get Foreclosure

In [5]:
table_name = "foreclosure_list"

mongod = mongo()


try:
    existing = mongod.conn.get_data("select * from Housing.%s "%table_name)

#     count = len(existing)
except:
    count = 0
    existing = pd.DataFrame()

count = 0

while count < 500:

    querystring = {
        "sort":"newest",
        "city":city,
        "offset":count,
        "state_code":"CA",
        "limit":200,
        "radius":20,
        "is_foreclosure":True


    }

    url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"


    headers = {
        'x-rapidapi-host': "realtor.p.rapidapi.com",
        'x-rapidapi-key': api_key
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    listing = response.json()

    listing= pd.DataFrame(listing["properties"])

    listing = listing[["property_id","listing_id","address","baths","baths_full","beds",\
                       "building_size","prop_type","last_update","lot_size",\
                        "rdc_web_url","price","page_no","client_display_flags"]]

    new_listing = pd.DataFrame()
    address = pd.DataFrame()
    building = pd.DataFrame()
    lot = pd.DataFrame()
    flags = pd.DataFrame()
    for i in listing.index:

        temp = listing.loc[i]
        if temp["lot_size"] == temp["lot_size"]:
            lot = lot.append(pd.DataFrame(temp["lot_size"],index =[temp.name]))
            temp = temp.drop("lot_size")
        try:
            del temp["address"]["neighborhoods"]
        except: 
            pass
        if temp["address"] == temp["address"]:
            address = address.append(pd.DataFrame(temp["address"],index =[temp.name]))
        temp = temp.drop("address")
        if temp["building_size"] == temp["building_size"]:
            building = building.append(pd.DataFrame(temp["building_size"],index =[temp.name]))
        temp = temp.drop("building_size")
        if temp["client_display_flags"] == temp["client_display_flags"]:
            flags = flags.append(pd.DataFrame(temp["client_display_flags"],index =[temp.name])[["is_new_listing","is_turbo","is_foreclosure","is_new_plan","has_open_house"]])
        temp = temp.drop("client_display_flags")

        new_listing = new_listing.append(temp)

  
    new_listing = new_listing.join(address).join(building,rsuffix="building_").join(lot,rsuffix="lot_").join(flags)

    def format_date(data):
        if data is None:
            return None
        try:
            return datetime.strptime(data,"%Y-%m-%dT%H:%M:%SZ")
        except:
            return datetime.strptime(data,"%Y-%m-%d")

    new_listing.last_update = new_listing.last_update.apply(format_date)

    new_listing["Refresh_Date"] = datetime.now()
    
    
    ### append new listing to existing  
    mongod = mongo()
    exist_count =len(existing)
    existing = existing.append(new_listing).drop_duplicates("property_id")

    ### update the table
    mongod = mongo()
    try:
        mongod.conn.conn.cursor().execute("drop table Housing.%s"%table_name)
    except:
        pass
    mongod.conn.to_sql(existing,database="Housing",table_name=table_name,if_exists="append")
    # get new count  
    
    if exist_count == len(existing):
        break
#     count = len(existing)

    count += 200
    


Connection Successful
Connection Successful
Connection Successful
If no error message, task completed


# For Sale

In [6]:
table_name = "for_sale_list"

mongod = mongo()


try:
    existing = mongod.conn.get_data("select * from Housing.%s "%table_name)

#     count = len(existing)
except:
    count = 0
    existing = pd.DataFrame()
count = 0
while count < 2000:

    querystring = {
        "sort":"newest",
        "city":city,
        "offset":count,
        "state_code":"CA",
        "limit":200,
        "radius":20,
        "is_foreclosure":False


    }

    url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"


    headers = {
        'x-rapidapi-host': "realtor.p.rapidapi.com",
        'x-rapidapi-key': api_key
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    listing = response.json()

    listing= pd.DataFrame(listing["properties"])

    listing = listing[["property_id","listing_id","address","baths","baths_full","beds",\
                       "building_size","prop_type","last_update","lot_size",\
                        "rdc_web_url","price","page_no","client_display_flags"]]

    new_listing = pd.DataFrame()
    address = pd.DataFrame()
    building = pd.DataFrame()
    lot = pd.DataFrame()
    flags = pd.DataFrame()
    for i in listing.index:

        temp = listing.loc[i]
        if temp["lot_size"] == temp["lot_size"]:
            lot = lot.append(pd.DataFrame(temp["lot_size"],index =[temp.name]))
            temp = temp.drop("lot_size")
        try:
            del temp["address"]["neighborhoods"]
        except: 
            pass
        if temp["address"] == temp["address"]:
            address = address.append(pd.DataFrame(temp["address"],index =[temp.name]))
        temp = temp.drop("address")
        if temp["building_size"] == temp["building_size"]:
            building = building.append(pd.DataFrame(temp["building_size"],index =[temp.name]))
        temp = temp.drop("building_size")
        if temp["client_display_flags"] == temp["client_display_flags"]:
            flags = flags.append(pd.DataFrame(temp["client_display_flags"],index =[temp.name])[["is_new_listing","is_turbo","is_new_plan","has_open_house"]])
        temp = temp.drop("client_display_flags")

        new_listing = new_listing.append(temp)

  
    new_listing = new_listing.join(address).join(building,rsuffix="building_").join(lot,rsuffix="lot_").join(flags)

    def format_date(data):
        if data is None:
            return None
        try:
            return datetime.strptime(data,"%Y-%m-%dT%H:%M:%SZ")
        except:
            return datetime.strptime(data,"%Y-%m-%d")

    new_listing.last_update = new_listing.last_update.apply(format_date)

    new_listing["Refresh_Date"] = datetime.now()
    
    
    ### append new listing to existing  
    mongod = mongo()
    exist_count = len(existing)
    existing = existing.append(new_listing).drop_duplicates("property_id")

    ### update the table
    mongod = mongo()
    try:
        mongod.conn.conn.cursor().execute("drop table Housing.%s"%table_name)
    except:
        pass
    mongod.conn.to_sql(existing,database="Housing",table_name=table_name,if_exists="append")
    # get new count  
    if exist_count == len(existing):
        break
    
#     count = len(existing)

    count += 200
    


Connection Successful
Connection Successful
Connection Successful
If no error message, task completed
Connection Successful
Connection Successful
If no error message, task completed
Connection Successful
Connection Successful
If no error message, task completed


In [34]:
def get_prop_detail(ID):

    querystring = {
        "property_id":str(ID)
    }

    url = "https://realtor.p.rapidapi.com/properties/v2/detail"

    headers = {
        'x-rapidapi-host': "realtor.p.rapidapi.com",
        'x-rapidapi-key': "L4vgCBGurvmshQp6E7hWIVXE2wyup1Ku0rGjsnjnhyKjfJeFwS"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    prop= pd.DataFrame(response.json()["properties"])
    
    prop = prop[["buyer_agent_for_far","buyer_office","description","property_history","tax_history","hoa_fee","sold_history","prop_status"]]
    return prop

In [38]:
get = get_prop_detail("M2171938907").iloc[0]

In [40]:
get["sold_history"] 

[{u'date': u'2009-02-20T07:00:00Z',
  u'listing': {u'price': 2200000},
  u'source': u'public record'}]

In [6]:
querystring = {
    "sort":"sold_date",
    "city":"concord, antioch",
    "offset":0,
    "state_code":"CA",
    "limit":200,
    "radius":20


}

url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"


headers = {
    'x-rapidapi-host': "realtor.p.rapidapi.com",
    'x-rapidapi-key': "93dedfc226msh5f733777e6fa72dp12da64jsneba592d0c8c8"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

listing = response.json()

listing= pd.DataFrame(listing["properties"])

# listing = listing[["property_id","listing_id","address","baths","baths_full","beds",\
#                    "building_size","last_update","list_date","lot_size",\
#                     "rdc_web_url","year_built","price","page_no"]]



In [15]:
mongod = mongo()

Connection Successful


In [16]:
data = pd.read_csv(directory+"sold_list_backup.csv")

In [17]:
mongod.conn.to_sql(data,database="Housing",table_name="sold_list")

If no error message, task completed


In [23]:
! python homeowner_auto_update.py

/home/ken/.local/lib/python2.7/site-packages/pyfolio/pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'
/home/ken/.local/lib/python2.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
Connection Successful
Connection Successful
If no error message, task completed
Connection Successful
If no error message, task completed
Connection Successful
If no error message, task completed
Connection Successful
Connection Successful
If no error message, task completed
Connection Successful
If no error message, task completed
Connection Successful
If no error message, task completed
Connection Successful
Connection Successful
If no error message, ta